<a href="https://colab.research.google.com/github/kumardesappan/colab-notebooks/blob/main/save_timm_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip3 install onnx-simplifier
!pip install timm
!pip install onnxruntime

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 20.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.9/15.9 MB 49.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 26.0 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_

In [2]:
import timm
import torch
from onnxsim import simplify
import onnx
from pathlib import Path
import os

#all_models = timm.list_models(pretrained=True)
# for model in all_models:
#     print(model)

cls_models = {"mobilenetv2_100.onnx":{"url":"mobilenetv2_100.ra_in1k"},
              #"resnet50.onnx":{"url":"resnet50.tv_in1k"},
              # "convnext_small.onnx":{"url":"convnext_small.fb_in1k"},
              # "regnetx_008.onnx":{"url":"regnetx_008.tv2_in1k"},
              # "efficientnet_lite0.onnx":{"url":"efficientnet_lite0.ra_in1k"},
            #  "deit_tiny_patch16_224.onnx":{"url":"deit_tiny_patch16_224.fb_in1k"},
            #   "efficientvit_b2.r224.onnx":{"url":"efficientvit_b2.r224_in1k"},
               "swin_small_patch4_window7_224.onnx":{"url":"swin_small_patch4_window7_224.ms_in1k"},
              }

for name in cls_models:
    model = timm.create_model(cls_models[name]["url"], pretrained=True)
    model.eval()
    base_dir = Path(name).stem
    os.mkdir(base_dir)
    file_name = os.path.join(base_dir, "network.onnx")

    dummy_input = torch.ones(1, 3, 224, 224, dtype=torch.float)
    torch.onnx.export(model, dummy_input, file_name, export_params=False, verbose=True,opset_version=11)
    model_org = onnx.load(file_name)
    # convert model
    model_simp, check = simplify(model_org)
    onnx.save(model_simp, file_name)



/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


model.safetensors:   0%|          | 0.00/14.2M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/200M [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/torch/__init__.py:1499: TracerWarning: Converting a tensor to a Python boolean might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
  assert condition, message
